# Process Corpus for Sentence extractions

1. Process the Dutch wiki xml dump.
2. Get rid of tags and other elements from the xml format. (Use perl script)
3. Retrieve text only from the corpus. 
4. Use nltk to split everything in sentences.

In [1]:
from gensim.corpora import WikiCorpus
from gensim.test.utils import datapath, get_tmpfile
# corpus_path = get_tmpfile('../Rodrigo-data/Corpus/nlwiki-corpus.mm')

unable to import 'smart_open.gcs', disabling that module


In [2]:
wiki = WikiCorpus('../Rodrigo-data/Corpus/wiki/nlwiki-20181001-corpus.xml.bz2')

FileNotFoundError: [Errno 2] No such file or directory: '../Rodrigo-data/Corpus/wiki/nlwiki-20181001-corpus.xml.bz2'

In [3]:
# print first 10 lines
with open('../Rodrigo-data/Corpus/nlwiki-20181001-corpus.xml') as f:
    for x in range(200):
        print(f.readline().strip())

FileNotFoundError: [Errno 2] No such file or directory: '../Rodrigo-data/Corpus/nlwiki-20181001-corpus.xml'

In [4]:
import subprocess
#call the perl script from the wiki-dump source https://linguatools.org/tools/corpora/wikipedia-monolingual-corpora/
#actual script is a local file
input_wiki = '../Rodrigo-data/Corpus/Wiki/nlwiki-20181001-corpus.xml'
out_wiki = "../Rodrigo-data/Corpus/Wiki/nl-wiki-perl-test.txt"
# pipe = subprocess.Popen(["perl", "../Rodrigo-data/Corpus/Wiki/xml2txt.pl" ,input_wiki , out_wiki], stdout=subprocess.PIPE)
#this wont block the script so wait and make sure the out file exists


In [5]:
#Use nltk to make the wiki text file in a list of sentences and write to file.
import nltk
nltk.download('punkt')
with open('../Rodrigo-data/Corpus/Wiki/nlwiki-perl.txt') as f:
    wikidumptext = f.read()
    list_sent = nltk.tokenize.sent_tokenize(wikidumptext,language='dutch')
    with open('../Rodrigo-data/Corpus/Wiki/lists_nlwiki-perl.txt', 'w') as filehandle:
        filehandle.writelines("%s\n" % sent for sent in list_sent)

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


KeyboardInterrupt: 

In [ ]:
# Check sentences in file:
with open('../Rodrigo-data/Corpus/Wiki/lists_nlwiki-perl.txt') as f:
    for x in range(50):
        print(f.readline().strip())

# Pairs Gender from Wiki dump

In [2]:
words = [["vrouw", "man"], ["meisje", "jongen"], ["zij", "hij"], ["moeder", "vader"], ["dochter", "zoon"], ["zus","broer"], ["vrouwelijk","mannelijk"], ["haarzelf","hemzelf"], ["Emma","Daan"]]

In [1]:
def match(a,L):
	for b in L:
		if a == b:
			return True
	return False

def replace(a,new,L):
	Lnew = []
	for b in L:
		if a == b:
			Lnew.append(new)
		else:
			Lnew.append(b)
	return ' '.join(Lnew)
# def check_duplicate(pair, pair_list):
# 	p0 = pair[0]
# 	p1 = pair[1]
# 	if p0 in pair_list or p1 in pair_list:
# 		return True
# 	else:
# 		return False
def template2(words, sent, sent_words_list, all_pairs):
	for (female, male) in words:
		if match(female, sent_words_list):
			sent_f = sent
			sent_m = replace(female,male,sent_words_list)
			pair = [sent_f,sent_m]
			# if not check_duplicate(pair, all_pairs):
			# 	all_pairs.append(pair)
			all_pairs.append(pair)
		if match(male, sent_words_list):
			sent_f = replace(male,female,sent_words_list)
			sent_m = sent
			pair_m = [sent_f,sent_m]
			# if not check_duplicate(pair, all_pairs):
				# all_pairs.append(pair)
			all_pairs.append(pair_m)
	return all_pairs

In [3]:
# Check sentences in file:
from tqdm import tqdm
wiki_lists = [] 
c=0
f_name = '../Rodrigo-data/Corpus/Wiki/lists_nlwiki-perl.txt'
num_lines = sum(1 for line in open(f_name,'r'))
with open(f_name) as f:
   for line in tqdm(f, total=num_lines):
        wiki_lists.append(line.strip())
        c+=1
        # if c>7533267 : break


100%|██████████| 37533267/37533267 [00:24<00:00, 1514492.33it/s]


In [4]:
len(wiki_lists)

37533267

In [5]:
wiki_lists[5]

'Het gezin waarin Speer werd geboren, was wat ze in het Duits noemen grossbürgerlich; in Nederlandse termen gegoede burgerij of bourgeoisie.'

In [6]:
allpairs=[]
count =0
for sentence in wiki_lists:
    count += 1
    if len(sentence)>490 or len(sentence)<90:
        # print(sentence)
        continue
    template2(words,sentence,sentence.split(' '),allpairs)
    if count%50000 == 0:
        print(count)
    if count > 11350000:
        break
# allpairs2 = set(allpairs)


450000
500000
800000
850000
900000
1050000
1100000
1150000
1300000
1400000
1500000
1600000
2250000
2450000
2550000
2800000
3550000
3650000
3750000
3800000
3900000
4250000
4400000
4850000
5050000
5650000
5700000
6300000
6350000
6800000
7100000
7200000
7250000
7300000
7400000
7500000
8250000
8400000
8500000
8600000
8700000
8900000
8950000
9050000
9100000
9450000
9650000
9800000
10350000
10450000
10500000
10600000
10750000
10800000
10900000
11300000


In [7]:
len(allpairs)

404321

In [8]:
import json
filename =  '../Rodrigo-data/gender-pairs/nl_gender_pairs_large_nonalpha.json' #changed 12july
# filename='nl_gender_pairs_large_nonalpha.json'
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(allpairs, f, ensure_ascii=False, indent=4)

In [9]:
with open(filename) as jsonfile:
    loaded_pairs = json.load(jsonfile)

In [10]:
len(loaded_pairs)

404321

In [11]:
loaded_pairs[5][0]
loaded_pairs[5][0]
first_col = [row[0] for row in loaded_pairs]   
second_col = [row[1] for row in loaded_pairs]
print(len(first_col))
print(len(second_col))
for i in range(len(second_col)):    
    if len(second_col[i]) <90:    
        print(first_col[i])
        print(second_col[i])

d.
Hij besluit uiteindelijk om Lennie te vermoorden, aangezien hij Curly's man heeft gedood.
Toen deze de plannen van Svjatopolk vernam, liet hij hem, zijn vrouw en Reinbern opsluiten.
Toen deze de plannen van Svjatopolk vernam, liet hij hem, zijn man en Reinbern opsluiten.
Aldus vroeg Hipparétê, vrouw van Alkibiades, persoonlijk echtscheiding aan bij de archont.
Aldus vroeg Hipparétê, man van Alkibiades, persoonlijk echtscheiding aan bij de archont.
Het verhaal draait om dichter Dylan Thomas en de relatie met zijn vrouw Caitlin MacNamara.
Het verhaal draait om dichter Dylan Thomas en de relatie met zijn man Caitlin MacNamara.
Zij was een dochter van Hendrik van Schotland, 3e graaf van Huntingdon en Ada van Warenne.
Zij was een zoon van Hendrik van Schotland, 3e graaf van Huntingdon en Ada van Warenne.
Daardoor speelde hij, na Rent en The Wild Party, wederom samen met zijn vrouw Idina Menzel.
Daardoor speelde hij, na Rent en The Wild Party, wederom samen met zijn man Idina Menzel.
Mari